In [34]:
import os
import numpy as np
import pandas as pd
import re
import copy

In [35]:
# 将从DRAMP数据库下载的数据合并，这些文件必须都包括这些列'DRAMP_ID','Name','N-terminal_Modification', 'Sequence', 'C-terminal_Modification', 'Other_Modifications', 'Target_Organism'，并将不成功合并的文件名输出
#下载的文件放入一个目录里，该目录下不要放其他文件
os.chdir(r'E:\文件\大学课程\大四上\毕业设计\数据收集\收集整理\后期数据清洗整理\DRAMP\网站下载')
path = os.getcwd()  # 获取当前路径
listDir = os.listdir(path)  # 获取当前目录下的所有内容
listDir=listDir[0:-1]         #如果报错FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\$Antibacterial_amps.xlsx'的话需要把无法读取的文件从listDir中删除

"""df作为一个中间暂存的数据表，需要不断叠加新的符合条件的xlsx文件"""
df = pd.DataFrame(columns=['DRAMP_ID','Name','N-terminal_Modification', 'Sequence', 'C-terminal_Modification', 'Other_Modifications', 'Target_Organism'])
for file in listDir:
        excel = pd.read_excel(file,header=0)
        excelcol = set(excel.columns)
        dfcol = set(df.columns)
        if dfcol.issubset(excelcol) == True:  #将具有这六列的数据加到df数据表中
            mdf = pd.DataFrame(columns=df.columns)
            for i in df.columns:
                mdf[i] = excel[i]
            df = pd.concat([df, mdf])
        else:
            print(file)
            # print(excelcol)     #不符合条件的文件得重新看缺了哪些信息
df.to_excel(r'E:\文件\大学课程\大四上\毕业设计\数据收集\收集整理\后期数据清洗整理\DRAMP\1_DRAMP下载合并文件.xlsx',index=False)

candidate_amps.xlsx
clinical_amps.xlsx
patent_amps.xlsx
plant_amps.xlsx
specific_amps.xlsx
stapled_amps.xlsx
synthetic_amps.xlsx


In [36]:
# 对数据表进行重复行删除操作
df = df.drop_duplicates()
df.to_excel(r'E:\文件\大学课程\大四上\毕业设计\数据收集\收集整理\后期数据清洗整理\DRAMP\2_DRAMP删除重复项.xlsx',index=False)   #输出文件方便查看每一步结果


In [37]:
df

,DRAMP_ID,Name,N-terminal_Modification,Sequence,C-terminal_Modification,Other_Modifications,Target_Organism
0,DRAMP00089,Bacteriocin E50-52 (Preclinical),Free,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Free,None,Gram-negative bacteria: Campylobacter jejuni (...
1,DRAMP00107,Bacteriocin L-1077,Free,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Free,None,Gram-negative bacteria: Salmonella Enteritidis...
2,DRAMP00136,Enterocin E-760 (Bacteriocin),Free,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Free,None,[Ref.18086839]Gram-positive bacteria: Listeria...
3,DRAMP00171,Lactocyclicin Q (Bacteriocin),Cyclization (N termini to C termini),LIDHLGAPRWAVDTILGAIAVGNLASWVLALVPGPGWAVKAGLATA...,Cyclization (C termini to N termini),The whole peptide has a cyclic structure in wh...,Gram-positive bacteria: Lactococcus sp.strain ...
4,DRAMP00191,Microcin J25 (MccJ25; Bacteriocin),Cyclization of a N-terminal between Gly and Glu,GGAGHVPEYFVGIGTPISFYG,Free,None,Gram-negative bacteria: Escherichia coli BM21 ...
...,...,...,...,...,...,...,...
4751,DRAMP02515,Cc-CATH1,Not included yet,NaN,Not included yet,Not included yet,No MICs found in DRAMP database
4772,DRAMP18425,"Ocellatin-PT6 (frog, amphibians, animals)",Not included yet,GVFDIIKGAGKQLIAHAMEKIAEKVGLNKDGN,Not included yet,Not included yet,No MICs found in DRAMP database
5088,DRAMP20770,"Naegleriapore B (NP-B; parasite, amoebozoa, pr...",Not included yet,SVIGCEICEWLVATAEGFVNKTKPQIEQELLQICAKLGPYEQICDQ...,Not included yet,Not included yet,No MICs found in DRAMP database
5151,DRAMP20835,StII1-30 StII peptide derivative,Free,ALAGTIIAGASLTFQVLDKVLEELGKVSRK,Free,None,No MICs found in DRAMP database


In [85]:
#定义一个将所有混合的targets混合形式输出成单个target并以列表的形式保存。
def Single_target(composed,sybol):
    composed=composed.split(sybol)
    targets=[]
    for target in composed:
        if '#' in target:
            new_targets=target.split('#')
            for i in new_targets:
                if '(' in i:
                    targets.append(i)
        elif '(' in target:
            targets.append(target)

    return targets


In [88]:
#把多个target数据分离成单个target在保持其他列数据不变的情况下追加成新行

os.chdir('E:\文件\大学课程\大四上\毕业设计\数据收集\收集整理\后期数据清洗整理\DRAMP')
df1=pd.read_excel(r'2_DRAMP删除重复项.xlsx')

newdf=pd.DataFrame(columns=df1.columns)   #新建一个数据表写入重构的数据
new_index=0
for index in range(len(list(df1['Target_Organism']))):
    targets=df1.at[index,'Target_Organism'].strip()   #获取混合的targets
    targets=targets.replace(';',',')
    targets=targets.replace('，',',')
    targets=targets.replace('and',',')
    if ',' in targets:
        for j in Single_target(targets,','):
            old_row=df1.loc[index].copy()
            old_row['Target_Organism']=j
            newdf.loc[new_index]=old_row
            new_index+=1
    elif '#' in targets and targets.count('(')>1:
        for j in Single_target(targets,'#'):
            old_row=df1.loc[index].copy()
            old_row['Target_Organism']=j
            newdf.loc[new_index]=old_row
            new_index+=1
    elif targets.count('(') == 1:
        old_row=df1.loc[index].copy()
        newdf.loc[new_index]=old_row
        new_index+=1
    else:
        pass

In [90]:
#将拆分好的文件输出成xlsx表格
newdf.to_excel('3_DRAMP拆分成单个target行.xlsx',index=False)

In [100]:
#定义一个修改value值的函数
def revise_value(value):
    value1 = str(value)       #使用替换的方法可以快速换掉错乱的字符
    value = value1.strip()
    value = value.replace('+/-', '@')
    value = value.replace('±', '@')
    value = value.replace('+/', '@')
    value = value.replace('+-', '@')
    value = value.replace('+', '@')
    value = value.replace('>', '')
    value = value.replace('<', '')
    value = value.replace('≥', '')
    value = value.replace('≤', '')
    value = value.replace('~', '')
    value = value.replace('=', '')
    value = value.replace(' ', '')
    value = value.replace(',', '.')
    value = value.replace('to', '-')
    value = value.replace('–', '-')
    value=value.strip()
    for bit in value:
        if bit.isalpha()==True:
            value=''
            break
    if '[' in value:
        value =''
    elif '×' in value:
        value = ''
    elif '?' in value:
        value=''
    elif '_' in value:
        value = ''
    elif '-' in value:
        values = value.split('-')
        num1 = float(values[0].strip())
        num2 = float(values[1].strip())
        value = (num1 + num2) / 2
    elif "@" in value:
        values = value.split('@')
        value = values[0].strip()
    return value

  #index默认为True时是会有一列索引

In [121]:
#创建一个函数用于把measure_value_unit这个字段拆分成三个字段measure、value、unite分别对应测算方法，数值，单位
def seprate_mics(mics):
    measure=''
    num=''
    unit=''
    mics=mics.replace('≤','=')
    mics=mics.replace('≥','=')
    if '=' in mics:
        mics = mics.split('=')
        measure = mics[0].strip()
        a = mics[1].strip().split(' ')
        if len(a) == 2:
            num = a[0]
            unit = a[1]
            # print(num+':'+unit)
            # measure_num_unnit=[measure,num,unit]
        elif len(a) == 1:
            # print(a)
            num = ''
            for j in a[0]:
                if j.isalpha() == False:
                    num += j
                elif j.isalpha() == True:
                    break
            unit = a[0].strip(num)
            # print(num + ':' + unit)
            # measure_num_unnit=[measure,num,unit]
        elif len(a) == 4:
            # print(a)
            num = a[0]+a[1]+a[2]
            unit = a[3]
            # print(num + ':' + unit)
            # measure_num_unnit=[measure,num,unit]
        elif len(a) == 3:
            if a[1] == '' or a[1] == ' ':
                num = a[0]
                unit = a[-1]
                # print(num + ':' + unit)
                # measure_num_unnit=[measure,num,unit]
            else:
                num = a[0]+a[1]
                unit = a[2]
                # print(num + ':' + unit)
                # print(a)
                # measure_num_unnit=[measure,num,unit]
        elif len(a) == 5:
            num = a[0]+a[1]+a[2]+a[3]
            unit = a[4]
            # print(num + ':' + unit)
            # measure_num_unnit=[measure,num,unit]
    elif '>' in mics:
            mics = mics.split('>')
            measure = mics[0].strip()
            a = mics[1].strip().split(' ')
            if len(a) == 2:
                num = '>'+a[0]
                unit = a[1]
                # print(num+':'+unit)
                # measure_num_unnit=[measure,num,unit]
            elif len(a) == 1:
                # print(a)
                num = ''
                for j in a[0]:
                    if j.isalpha() == False:
                        num += j
                    elif j.isalpha() == True:
                        break
                num = '>' + num
                unit = a[0].strip(num)
                # print(num + ':' + unit)
                # measure_num_unnit=[measure,num,unit]
            elif len(a) == 4:
                # print(a)
                num = '>' + a[0] + a[1] + a[2]
                unit = a[3]
                # print(num + ':' + unit)
                # measure_num_unnit=[measure,num,unit]
            elif len(a) == 3:
                if a[1] == '' or a[1] == ' ':
                    num = '>'+a[0]
                    unit = a[-1]
                    # print(num + ':' + unit)
                    # measure_num_unnit=[measure,num,unit]
                else:
                    num = '>' + a[0] + a[1]
                    unit = a[2]
                    # print(num + ':' + unit)
                    # print(a)
                    # measure_num_unnit=[measure,num,unit]
            elif len(a) == 5:
                num = '>'+a[0] + a[1] + a[2] + a[3]
                unit = a[4]
                # print(num + ':' + unit)
                # measure_num_unnit=[measure,num,unit]
    elif '<' in mics:
        mics = mics.split('<')
        measure = mics[0].strip()
        a = mics[1].strip().split(' ')
        if len(a) == 2:
            num = '<' + a[0]
            unit = a[1]
            # print(num+':'+unit)
            # measure_num_unnit=[measure,num,unit]
        elif len(a) == 1:
            # print(a)
            num = ''
            for j in a[0]:
                if j.isalpha() == False:
                    num += j
                elif j.isalpha() == True:
                    break
            num = '<' + num
            unit = a[0].strip(num)
            # print(num + ':' + unit)
            # measure_num_unnit=[measure,num,unit]
        elif len(a) == 4:
            # print(a)
            num = '<' + a[0] + a[1] + a[2]
            unit = a[3]
            # print(num + ':' + unit)
            # measure_num_unnit=[measure,num,unit]
        elif len(a) == 3:
            if a[1] == '' or a[1] == ' ':
                num = '<' + a[0]
                unit = a[-1]
            else:
                num = '<' + a[0] + a[1]
                unit = a[2]
        elif len(a) == 5:
            num = '<' + a[0] + a[1] + a[2] + a[3]
            unit = a[4]
            measure_num_unit=[measure,num,unit]
    num = revise_value(num)
    return [measure,num,unit]

# 定义一个删去unit多余后缀的函数
def Unit(unit):
    unit=unit.replace('.','')
    unit=unit.replace(')','')
    unit=unit.replace(';','')
    unit=unit.replace('；','')
    return unit

In [123]:
# 将target_activity这一列数据拆分成四列：target，measure，value，unit
newdf2=copy.deepcopy(newdf )   #创建一个过渡数据表，防止修改掉之前形成的数据表
new_columns=['target','measure','value','unit']
for column in new_columns:
    newdf[column]=np.NaN
# newdf2.head()
# target_list=[]
for i in range(len(list(newdf['Sequence']))):
    targets_mics=newdf2.at[i,'Target_Organism']
    if targets_mics.count('(')==1:
        composed=targets_mics.split('(')
        # target=composed[0].strip()
        newdf2.at[i,'target']=composed[0].strip()
        micss=composed[1].strip(')')
        miclist=seprate_mics(micss)
        newdf2.at[i,'measure']=miclist[0]
        newdf2.at[i,'value']=miclist[1]
        newdf2.at[i,'unit']=Unit(miclist[2])
    elif targets_mics.count('(')==2:
        composed=targets_mics.split('(')
        newdf2.at[i,'target']=(composed[0]+composed[1]+')').strip()
        micss=composed[-1].strip(')')
        miclist=seprate_mics(micss)
        newdf2.at[i,'measure']=miclist[0]
        newdf2.at[i,'value']=miclist[1]
        newdf2.at[i,'unit']=Unit(miclist[2])
    else:
        pass

C:\Users\lanliu\AppData\Local\Temp\ipykernel_13564\3576525282.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  newdf2.replace('', pd.np.nan, inplace=True)
C:\Users\lanliu\AppData\Local\Temp\ipykernel_13564\3576525282.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['database']='DRAMP'
C:\Users\lanliu\AppData\Local\Temp\ipykernel_13564\3576525282.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.rename(columns=new_columns, inplace=True)


In [126]:
# 删除value列中存在空字符或者空白记录的字段，并把value列改为数字格式
new=copy.deepcopy(newdf2)
new['value'] = pd.to_numeric(new['value'], errors='coerce')
new.dropna(subset=['value'], inplace=True)
# new['value'] = new['value'].astype(float)
# 添加数据库来源标识
new['database']='DRAMP'
# 对表格修改列名
new_columns={'DRAMP_ID':'database_id','Name':'name','N-terminal_Modification':'n_terminal',
             'Sequence':'sequence','C-terminal_Modification':'c_terminal','Other_Modifications':'other_modification'}
new.rename(columns=new_columns, inplace=True)
new.to_excel(r'4_DRAMP拆分mic.xlsx',index=False)


In [127]:
finaldata=new.drop('Target_Organism', axis=1)
finaldata.to_excel(r'DRAMP.xlsx',index=False)